!pip install -U numpy

import sys
print(sys.path) 
print(sys.executable)

import sys 
!{sys.executable} -m pip install nltk

# Import Packages

In [8]:
import pandas as pd
import numpy as np 
import csv
import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings("ignore")


# Dataset Cleaning

In [9]:
path = '/home/manikya_varshney/Documents/Python/Yale/Data/h01-20200912-101538.csv'
data = pd.read_csv(path, low_memory=False)

In [ ]:
data.shape

In [10]:
cols = [1, 3, 4, 5,6, 7, 8, 9, 10, 11, 17, 18, 19, 22, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 45, 54, 55, 56, 58, 63, 64, 65, 67, 76, 77]

In [11]:
data = data[data.columns.values[cols]]

In [ ]:
data.shape

In [12]:
data.to_csv('data_filtered_h01-20200912-101538.csv',index=False)

In [ ]:
data['lang'].value_counts()

In [ ]:
data[data['lang'] == 'en'].shape

In [13]:
data_en = data[data['lang'] == 'en'].reset_index(drop=True)

In [ ]:
data_en

In [14]:
data_en['RT_Flag'] = data_en['text'].str[:2]=='RT'

In [20]:
data_en.shape

(10043, 39)

### is_quote_tweet --> False --> Non RT

In [21]:
interim = data_en[np.where((data_en['is_quote_tweet']==False) & (data_en['RT_Flag']==False),True,False)].reset_index(drop=True)

In [30]:
interim['extended_tweet_full_text'] = np.where(~interim['truncated'],interim['text'],interim['extended_tweet_full_text'])

In [37]:
non_rep = interim[['id','user_id','extended_tweet_full_text']]

In [38]:
non_rep.shape

(2288, 3)

In [52]:
non_rep.drop_duplicates(['id']).shape

(2287, 3)

In [55]:
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

is_quote_tweet --> False --> RT

In [39]:
interim = data_en[np.where((data_en['is_quote_tweet']== False) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

In [40]:
interim.shape

(5182, 39)

In [42]:
interim[['RT_text','RT_full_text']]

,RT_text,RT_full_text
0,"Let me get this right, it was okay for Steven ...","Let me get this right, it was okay for Steven ..."
1,Hospitals in the United States have turned dow...,Hospitals in the United States have turned dow...
2,My friends parents have died from Covid &amp; ...,My friends parents have died from Covid &amp; ...
3,📢 Our 7-part series on life after lockdown ‘EM...,📢 Our 7-part series on life after lockdown ‘EM...
4,@JoeTalkShow I wrote a tweet after u came back...,@JoeTalkShow I wrote a tweet after u came back...
...,...,...
5177,A Perth woman in quarantine has become the fir...,A Perth woman in quarantine has become the fir...
5178,Have you struggled with your eating habits sin...,NaN
5179,I looked up the definition of corona in the Ch...,NaN
5180,"a stay at home challenge to keep people safe, ...","a stay at home challenge to keep people safe, ..."


In [44]:
interim['RT_full_text'] = np.where(interim['RT_full_text'].isna(),interim['RT_text'],interim['RT_full_text'])

In [57]:
interim = interim[['RT_id','RT_user_id','RT_full_text']]

In [60]:
interim.columns = non_rep.columns

Index(['id', 'user_id', 'extended_tweet_full_text'], dtype='object')

In [62]:
interim.shape

(5182, 3)

In [66]:
interim = interim.drop_duplicates(['id','user_id']).reset_index(drop=True)

In [73]:
non_rep = non_rep.append(interim,ignore_index=True)

In [75]:
non_rep.shape

(4508, 3)

In [80]:
non_rep['id'].value_counts()

1.304786e+18    2
1.304786e+18    2
1.304786e+18    2
1.304786e+18    2
1.304786e+18    2
               ..
1.304787e+18    1
1.304786e+18    1
1.304786e+18    1
1.304787e+18    1
1.304705e+18    1
Name: id, Length: 4444, dtype: int64

In [82]:
non_rep = non_rep.drop_duplicates(['id']).reset_index(drop=True)

is_quote_tweet --> True --> Non RT

In [17]:
data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== False),True,False)].reset_index(drop=True)

,id,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,reply_count,retweet_count,...,RT_full_text,QT_id,QT_lang,QT_user_id,QT_username,QT_name,QT_location,QT_text,QT_full_text,RT_Flag
0,1.304786e+18,True,1.304732e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304732e+18,en,1.074866e+18,stfuravenn,ʀᴀᴠᴇɴ,Philippines,R.I.P to my unhealthy phone charging habit,NaN,False
1,1.304786e+18,True,1.304775e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304775e+18,en,2.508580e+07,iTweetDaPUSSY,DONZELL 🦈,The Bronx ✈️ Souf Cak,So does mask or a face shield work better 🤣,NaN,False
2,1.304786e+18,True,1.304543e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304543e+18,en,4.163452e+07,seanhannity,Sean Hannity,NYC,BERNIE BUCKS: Sanders Says ‘Every Working Clas...,NaN,False
3,1.304786e+18,True,1.304783e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304783e+18,en,4.765364e+09,ChuckCallesto,Chuck Callesto,"Florida, USA",Do you think some STATE SHUTDOWNS are directly...,NaN,False
4,1.304786e+18,True,1.304774e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304774e+18,en,1.113425e+07,GOP,GOP,NaN,“The RNC says it has knocked on the doors of 1...,“The RNC says it has knocked on the doors of 1...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,1.304787e+18,True,1.304772e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304772e+18,en,1.568751e+07,BBCScotlandNews,BBC Scotland News,"Pacific Quay, Glasgow",BREAKING: 221 people have tested positive for ...,BREAKING: 221 people have tested positive for ...,False
754,1.304787e+18,True,1.304775e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304775e+18,in,3.254877e+09,kodimbardo1,Dr(C).Zoya Melani SH. MH,"Jakarta Pusat, DKI Jakarta",Kalau tidak py wacana perbaikan soal penangana...,Kalau tidak py wacana perbaikan soal penangana...,False
755,1.304787e+18,True,1.304754e+18,NaN,34116377.0,ScottMorrisonMP,en,0,0,0,...,NaN,1.304754e+18,en,1.292287e+18,YellowCube7,BeachMilk,NaN,This is what’s occurring in Australia @ScottMo...,This is what’s occurring in Australia @ScottMo...,False
756,1.304787e+18,True,1.304783e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304783e+18,en,1.192550e+18,GovAndyBeshear,Governor Andy Beshear,"Frankfort, Kentucky",Every life matters and this virus is our enemy...,Every life matters and this virus is our enemy...,False


is_quote_tweet --> True --> RT

In [18]:
data_en[np.where((data_en['is_quote_tweet']== True) & (data_en['RT_Flag']== True),True,False)].reset_index(drop=True)

,id,is_quote_tweet,quoted_tweet_id,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,lang,quote_count,reply_count,retweet_count,...,RT_full_text,QT_id,QT_lang,QT_user_id,QT_username,QT_name,QT_location,QT_text,QT_full_text,RT_Flag
0,1.304786e+18,True,1.304631e+18,NaN,NaN,NaN,en,0,0,0,...,This is why many of us have cautioned from the...,1.304631e+18,en,8.312794e+17,Craig_A_Spencer,Craig Spencer MD MPH,New York City,A radiologist with ZERO credentials for this r...,A radiologist with ZERO credentials for this r...,True
1,1.304786e+18,True,1.304568e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304568e+18,en,2.949916e+08,MindfulMajority,Mindful Majority,"Denver, CO",Joe Biden is mobilizing an impressive team of ...,Joe Biden is mobilizing an impressive team of ...,True
2,1.304786e+18,True,1.304603e+18,NaN,NaN,NaN,en,0,0,0,...,"Folks, literally for week ending Sep. 5, the P...",1.304603e+18,en,4.445154e+07,kylamb8,Kyle Lamb,"Columbus, OH",Weekly Covid Indicators (Covid Indicator Trend...,Weekly Covid Indicators (Covid Indicator Trend...,True
3,1.304786e+18,True,1.304626e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304626e+18,en,3.287109e+07,kylegriffin1,Kyle Griffin,"Manhattan, NY",TORONTO (Reuters) - Canada reported zero COVID...,TORONTO (Reuters) - Canada reported zero COVID...,True
4,1.304786e+18,True,1.304341e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304341e+18,en,7.585442e+07,konkonas,Konkona Sensharma,India,What an irresponsible statement! Corona cases ...,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810,1.304787e+18,True,1.304503e+18,NaN,NaN,NaN,en,0,0,0,...,NaN,1.304503e+18,en,1.501249e+07,CBSNews,CBS News,"New York, NY","WATCH: While hosting a large house party, Miam...","WATCH: While hosting a large house party, Miam...",True
1811,1.304787e+18,True,1.304756e+18,NaN,NaN,NaN,en,0,0,0,...,Completely forgot about the Louisville escorts...,1.304756e+18,en,2.215891e+09,LouClinicEscort,LvilleClinicEscorts,NaN,65 Catholics across the street only 7 wearing ...,NaN,True
1812,1.304787e+18,True,1.304610e+18,NaN,NaN,NaN,en,0,0,0,...,"Good morning. As 1,000+ Americans die a day, a...",1.304610e+18,en,1.975016e+07,justinhendrix,Justin Hendrix,NYC,Politico exclusive: Trump political appointees...,Politico exclusive: Trump political appointees...,True
1813,1.304787e+18,True,1.304093e+18,NaN,NaN,NaN,en,0,0,0,...,Fun fact: in 2008 this officer pistol whipped ...,1.304093e+18,en,8.863983e+17,davenewworld_2,Fifty Shades of Whey,He/Him — NYC-LI,Police officer in Pittsburgh arrests a man for...,NaN,True


# Analysis

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
def load_dataset(path, cols):
    df = pd.read_csv(path, low_memory=False)
    df.coloumns = cols
    return df

In [ ]:
def delete_cols(df, cols):
    for col in cols